<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/src/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and dependencies
Run the below cell, which shall clone the repo contents and prepare the machine for work. This may take a few minutes.

***Important:*** This is the first cell you shall run in this project, otherwise it won't be working.

After 'pip' is done, it may ask you to restart this session - please do accordingly.

In [1]:
# remove possible leftovers and clear-clone the whole repo
!cd /content
!rm -rf ./machine_learning_ev_ice
!git clone https://github.com/stas420/machine_learning_ev_ice.git

# a git pull pro forma
%cd ./machine_learning_ev_ice
!git pull

# go to our main directory
%cd ./src

# check (and download, if needed) dependencies for the project
%pip install -r requirements.txt

Cloning into 'machine_learning_ev_ice'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 53 (delta 21), reused 35 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 13.38 KiB | 13.38 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/machine_learning_ev_ice
Already up to date.
/content/machine_learning_ev_ice/src
  Using cached labelImg-1.8.6-py2.py3-none-any.whl
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached ultralytics-8.3.140-py3-none-any.whl.metadata (37 kB)
  Using cached PyQt5-5.15.11-cp38-abi3-manylinux_2_17_x86_64.whl.metadata (2.1 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manyli

# test run to check labelImg

In [2]:
# Launch labelImg
!cat ./requirements.txt

# LabelImg dependencies
setuptools == 78.1.0
labelImg == 1.8.6

# PyTorch
torch == 2.6.0
numpy == 2.2.4
ultralytics==8.3.140
opencv-python-headless==4.11.0.86
tqdm==4.67.1


# extrakt frejms

In [ ]:
import cv2
import os

def extract_frames(video_path, output_dir, fps=1):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(video_fps // fps)

    count = 0
    saved = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved:03d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved += 1
        count += 1
    cap.release()

# Przykład użycia
for x in os.listdir('.'):
  extract_frames(x, "imgs/" + x, fps=1)


# is grin plejt

In [ ]:
from ultralytics import YOLO
import cv2
import os

model = YOLO("models/license_plate.pt")
input_dir = "data/frames"
output_dir = "results/detected"
os.makedirs(output_dir, exist_ok=True)

def is_green_plate(plate_img):
    hsv = cv2.cvtColor(plate_img, cv2.COLOR_BGR2HSV)
    h, s, _ = hsv.mean(axis=0).mean(axis=0)
    return 35 < h < 85 and s > 40

for fname in os.listdir(input_dir):
    if not fname.endswith(".jpg"):
        continue
    path = os.path.join(input_dir, fname)
    img = cv2.imread(path)
    results = model(img)

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        crop = img[y1:y2, x1:x2]
        label = "ZIELONA" if is_green_plate(crop) else "Zwykła"
        color = (0,255,0) if label == "ZIELONA" else (255,0,0)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imwrite(os.path.join(output_dir, fname), img)


# prawdziwe mlowanie


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# 2. Wczytanie modelu YOLO (np. wytrenowany na tablicach)
model = YOLO("../models/license_plate.pt")

# 3. Wczytanie przykładowego obrazu
image_path = "../data/frames/frame_01.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 4. Detekcja tablic rejestracyjnych
results = model(image)

# 5. Analiza i klasyfikacja kolorów tablic
for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped = image[y1:y2, x1:x2]

    # Analiza koloru – przejście na HSV
    hsv = cv2.cvtColor(cropped, cv2.COLOR_BGR2HSV)
    avg_color = hsv.mean(axis=0).mean(axis=0)

    # Detekcja zielonej barwy
    h, s, v = avg_color
    is_green = 35 < h < 85 and s > 40

    color_label = "ZIELONA (elektryczny)" if is_green else "Zwykła"

    # Rysowanie wyników
    cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0) if is_green else (255,0,0), 2)
    cv2.putText(image, color_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                (0,255,0) if is_green else (255,0,0), 2)

# 6. Wyświetlenie wyników
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()


tensor([[0.9320, 0.5988, 0.1856],
        [0.5191, 0.3469, 0.9685],
        [0.1125, 0.9076, 0.0556],
        [0.0212, 0.8824, 0.7880],
        [0.8299, 0.5203, 0.5885]])


Dżem dobry
:)